# 数据连接与筛选

筛选北京数据

In [1]:
import geopandas as gpd
import matplotlib as plt
import pandas as pd
import os, glob

## 一、遍历文件


### 1）合并id对应用户属性excel→csv

In [6]:
# 文件夹的路径
from_path = r"E:\Netease\智慧城市\1.sixfoot全部相关数据\六只脚基本数据\【六只脚初始数据备份】（未给投影）\basic"
to_path = r"E:\Netease\智慧城市\4.处理数据\rawdata"

# 获取所有excel文件的列表
filenames = glob.glob(from_path + "\\*.xlsx")

# 初始化一个空的数据框
finalexcelsheet = pd.DataFrame()

# 遍历每个excel文件
for file in filenames[:2]:
    # 将每个excel文件中的所有工作表合并为一个数据框
    df = pd.concat(pd.read_excel(file, sheet_name=None), ignore_index=True, sort=False)
    # 将其添加到最终的数据框中
    finalexcelsheet = pd.concat([finalexcelsheet,df], ignore_index=True)

# 输出为一个新的csv文件
finalexcelsheet.to_csv(to_path+r'\行程信息.csv', index=False, encoding="utf-8_sig")

### 2）获得id对应线数据列表ls

In [2]:
folder =  r"E:\Netease\智慧城市\1.sixfoot全部相关数据\六只脚基本数据\【六只脚初始数据备份】（未给投影）"    #输入地址
outfolder = r"E:\Netease\智慧城市\4.处理数据"
datadirs = []

for root,dirs,files in os.walk(folder, topdown=False):
    for name in dirs: #遍历文件夹
        sigdir = os.path.join(root,name)
        
        pt = glob.glob(sigdir + r'\*track*.shp')
        datadirs.append(pt)

from itertools import chain

obj = chain.from_iterable(datadirs)
ls = list(obj)
print(len(ls))

32


## 二、数据处理：筛选+连接

In [3]:
track_zj = pd.read_csv(outfolder+r"\rawdata\行程信息.csv") #用户属性→赋值
track_zj["tripid"] = track_zj["tripid"].astype("str")
pl_bj = gpd.read_file(outfolder+r"\rawdata\北京市行政边界.shp") #北京行政边界→筛选
pl_bjqx = gpd.read_file(outfolder+r"\rawdata\北京市区县边界.shp").to_crs(epsg = 4326) #北京区县边界→赋值

In [ ]:
for a,i in enumerate(ls):
    print(a,i)
    track = gpd.read_file(i) #读取轨迹文件
    dt = track.merge(track_zj,on = 'tripid') #连接：通过tripid字段连接track与track_zj图层
    if dt.crs is None:
        dt = dt.set_crs(epsg = 4326)
    else:
        dt = dt.to_crs(epsg = 4326)
    
    dt_bj = gpd.overlay(dt,pl_bj,how = 'intersection',keep_geom_type=False) #筛选：图层取交集
    track = gpd.sjoin(dt_bj,pl_bjqx,predicate = 'intersects') #赋值：按空间关系将面信息赋给线（只要相交）
    track = track[['tripid', 'location', 'title', 'triptime', 'triptype', 'geometry']] #选择图层字段
    print(track.head())
    if track.empty is True:
        continue
    else:
        track.to_file(outfolder+r'\workingdata\track_zj_'+str(a)+'.shp',encoding = 'utf-8') #图层导出shp

### 筛选代码测试

In [ ]:
pl_bj.plot()

In [ ]:
track = gpd.read_file(r'E:\Netease\智慧城市\1.sixfoot全部相关数据\六只脚基本数据\【六只脚初始数据备份】（未给投影）\1_300000已完成（除去60000_160000，494_15494）\2_492+16040_60000+160000_222080\track2_492+16040_60000+160000_222080.shp')
print(track.head())

In [ ]:
track_zj = pd.read_csv(outfolder+r"\rawdata\行程信息.csv")
print(track_zj.head())

## 三、数据按出行类型分类

In [8]:
work_path = r"E:\Netease\智慧城市\2.相关代码及说明\轨迹数据清洗代码及说明(胡宝生)\轨迹数据分类(可能不完整，需核验)"
type_lst = os.listdir(work_path)

sum = 0
for type_file in type_lst:
    num = len(pd.read_csv(work_path+"\\"+type_file,encoding="gbk"))
    sum += num
    print(type_file,num)

print(sum)

bycicle_all.csv 11940
climbing_all.csv 64092
driving_all.csv 29723
moto_all.csv 892
others_all.csv 6653
running_all.csv 3409
sightsee_all.csv 15366
walk_all.csv 89579
221654
